# README.ipynb

Welcome to `labnotebook`. This is a docker-ized JupyterHub deployment that is setup to allow notebook kernels to run in Docker containers.

## Questions:

### Where should I store my data?

By default every user is granted a home directory at `/home/[username]`. Users in the `lab` group have read-only access to every other members' home directory. A shared folder at `/home/shared` can be written to by everyone, except the subdirectory `/home/shared/readonly` that requires `root` privliges to write to. The `/home/shared` directory is symlinked to each users' home directory for convenience at `/home/[username]/shared`.

Generally users should keep their own code in their home directory and shared source data in `/home/shared/readonly` to prevent it from being accidentally modified.

### Where should I store my kernels?

Some basic example kernel dockerfiles are provided in `/home/shared/kernels` and they can be built from that directory if desired. But I've found it helpful to create a new kernel definition for a project, and create a notebook describing it to store in the project directory: `/home/username/myproject/myprojectkernel.ipynb`. Any updates to the kernel definition can then be tracked by project version control. Letting all notebooks in a project use the same kernel definition makes it much easier to deploy.

### How can I grant `admin` privliges to a different user?

Admin privlidges consist of administration on the Hub, and `sudo` root access on the Docker system. Once the user account has been created (they must successfully log in after being Authorized by the `admin` user) then the `admin` user can choose File > Hub Control Panel > Admin > Edit User and then check the `admin` option. This will give that user the ability to authorize or decline new users to use the Hub.

Separately, the `admin` user can grant sudo privliges to `anotheruser` in a terminal by running:
```
sudo usermod -aG sudo anotheruser
```
In order to get this to work `anotheruser` will need to restart their server from the Hub Control Panel.

### How can I generate a PDF of a notebook?

Use `jupyter nbconvert`. See [shared/samples/Exporting to PDF.ipynb](/hub/user-redirect/lab/tree/home/admin/shared/samples/Exporting%20to%20PDF.ipynb) for an example.

### Is there a way to use version control?

Yes, the excellent [jupyterlab-git](https://github.com/jupyterlab/jupyterlab-git) extension is installed. To connect to a remote repository (eg. GitHub), you'll need to provide SSH keys. These are automatically generated for users at `~/.ssh/id_ed25519.pub`.

### Can I use another user's kernel container image?

Yes! Each user manages which containers are installed as kernels for them, but the images are globally available. To install a kernel all you need is the name of the docker container, and run the `!dockernel install ...` cell of the kernel definition. For example this cell will install the container image tagged as `tensorflow-2.13-gpu:latest` to the current user's kernelspec:

```
!dockernel install tensorflow-2.13-gpu:latest \
    --name TensorFlow-2.13-GPU \
    -v labnotebook-homedirs:/home:rw \
    --gpus all \
    --network host \
    --user -1 \
    --group-add -1 \
    --language python
```

Remember that since the image names are global, if you're modifying someone else's Dockerfile make sure you `docker build` with a different tag, or their kernel will start using your image. You can inspect what kernel container images are already built and available by running:

In [3]:
!docker images

REPOSITORY            TAG       IMAGE ID       CREATED        SIZE
scipy-notebook        latest    d03e1525b8af   13 hours ago   4.19GB
labnotebook           latest    03f7532fdf4d   13 hours ago   4.01GB
r-kernel              latest    d70e23314336   14 hours ago   2.83GB
tensorflow-2.13-gpu   latest    fd2816d45419   24 hours ago   6.67GB
matlab                latest    5646975c5629   37 hours ago   15GB
anaconda              latest    3aadca4f646b   40 hours ago   4.96GB


Note that this includes all docker images available, including the image running the labnotebook server, and any unrelated images on the machine.

### How do I remove a kernel?

First, list what kernels are installed with:

In [8]:
!jupyter kernelspec list

Available kernels:
  tensorflow-2.13-gpu-latest    /home/admin/.local/share/jupyter/kernels/tensorflow-2.13-gpu-latest
  python3                       /opt/conda/envs/jupyterenv/share/jupyter/kernels/python3


Users can remove a kernel by using:

```
!jupyter kernelspec remove [kernel-name]
```